# Deep Reinforcement Learning on Stock Trading (CS230 Milestone)

## Metadata
Author: Bicheng Wang, Xinyi Zhang   
Email: bichengw@stanford.edu, xyzh@stanford.edu


## Introdution


Profitable automated trading strategy plays a critical role to investment companies and hedge funds. Given that the stock market is dynamic and complex, it is challenging to design such a strategy. The project proposes to use a deep reinforcement learning framework to learn a profitable stock trading mechanism, with the goal to optimize the cumulative return and Alpha.
It would select S\&P 500 Index along with its top 20 market capitalization stocks as our trading stock pool.
The input to our algorithm is the market price for these stocks, remaining balance, current portfolio and technical indicator statistics. The model agent output is a series of trading actions among stocks. 
The available trading action options are: sell, buy and hold. 

## Dataset

We choose S\&P500 Index and 20 stocks with top performance in the index (exclude companies IPO after 2000) from 2000 to 2020 as our dataset. The original data is fetched from Yahoo Finance API. Each dataset row is comprised of date, open price, high price, low price, close price, volumn, ticker symbol, # day in a week and technical indicators. The technical indicators are computed using the original data, which include moving average convergence divergence, Bollinger Bands, Relative Strength Index, Commodity Channel Index , Directional Movement Index and Simple Moving Average.  The dataset has 105680 rows in total. We split the dataset into training and test on a 90/10 basis. The training set contains data ranging from 2000 to 2018, while the test set contains data ranging from 2019 to 2020.

## Implementation

Add model description, training stragety

## Results


Add early stage results

## Future plan

Add expected improvement 